In [1]:
import cv2
import numpy as np
import pickle
from skimage.feature import hog

In [2]:
with open('hog_svm_pipeline.pkl', 'rb') as model_file:
   hog_svm_model = pickle.load(model_file)

In [3]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [4]:
emotion_labels = ['anger', 'contempt', 'disgust', 'fear', 'happy', 'sadness', 'surprise']

emotion_label_to_id = {label: id for id, label in enumerate(emotion_labels)}

In [ ]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        face_roi = gray[y:y+h, x:x+w]

        face_roi = cv2.resize(face_roi, (84, 128))

        hog_features, svm_model = hog_svm_model
        hog_features = hog(face_roi, pixels_per_cell=(8, 8), cells_per_block=(2, 2))

        predicted_emotion_label = svm_model.predict([hog_features])[0]

        emotion_id = emotion_label_to_id.get(predicted_emotion_label, -1)

        cv2.putText(frame, emotion_labels[emotion_id], (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)

    cv2.imshow('Facial Expression Recognition', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()